In [2]:
import numpy as np
import pandas as pd
import cv2
import nibabel as nib
from pathlib import Path
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam

In [3]:
drive.mount("/content/drive")
heart_mri_path = Path("/content/drive/MyDrive/Heart Segmentation Data/imagesTr")
heart_label_path = Path("/content/drive/MyDrive/Heart Segmentation Data/labelsTr")

In [4]:
mri_files = list(heart_mri_path.glob("*.nii"))
label_files = list(heart_label_path.glob("*.nii"))


In [5]:
heart_data_mapping = pd.DataFrame({
    "MRI_IMAGE": [str(img) for img in mri_files],
    "LABEL_IMAGE": [str(lbl) for lbl in label_files]
})


In [6]:
processed_mri_images = []
processed_label_images = []

In [7]:
for mri_path, label_path in zip(heart_data_mapping["MRI_IMAGE"], heart_data_mapping["LABEL_IMAGE"]):
    mri_volume = nib.load(mri_path).get_fdata()
    label_volume = nib.load(label_path).get_fdata()

    middle_mri_slice = mri_volume[mri_volume.shape[0] // 2] / 255.0
    middle_label_slice = label_volume[mri_volume.shape[0] // 2] / 255.0

    if np.mean(middle_label_slice) > 0:
        resized_mri_slice = cv2.resize(middle_mri_slice.astype("float32"), (256, 256))
        resized_label_slice = cv2.resize(middle_label_slice.astype("float32"), (256, 256))

        processed_mri_images.append(resized_mri_slice)
        processed_label_images.append(resized_label_slice)

In [8]:
training_mri_images = np.array(processed_mri_images, dtype="float32").reshape(-1, 256, 256, 1)
training_label_images = np.array(processed_label_images, dtype="float32").reshape(-1, 256, 256, 1)

In [9]:
input_layer = Input(shape=(256, 256, 1))

In [10]:
x = Conv2D(32, (5, 5), kernel_initializer='he_normal')(input_layer)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv2D(64, (5, 5), kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv2D(128, (2, 2), kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
encoded = Conv2D(128, (2, 2), kernel_initializer='he_normal')(x)
encoded = BatchNormalization()(encoded)
encoded = ReLU()(encoded)

In [11]:
x = Conv2DTranspose(128, (2, 2))(encoded)
x = ReLU()(x)
x = Conv2DTranspose(64, (2, 2))(x)
x = ReLU()(x)
x = Conv2DTranspose(32, (5, 5))(x)
x = ReLU()(x)
decoded = Conv2DTranspose(1, (5, 5))(x)


In [12]:
tsa_model = Model(inputs=input_layer, outputs=decoded)


In [13]:
tsa_model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-7), metrics=["accuracy"])


In [ ]:
model_training_history = tsa_model.fit(training_mri_images, training_label_images, epochs=100)


In [ ]:
tsa_model.save('tsaModel.h5')
